### In this notebook, a Convolutional Neural Network is trained on Dogs vs Cats dataset from Kaggle 

#### https://www.kaggle.com/c/dogs-vs-cats

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import os
import shutil 
from tqdm import tqdm
# Keras is the Deep Learning library built on top on Tensorflow
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [2]:
print(os.listdir("../input"))
os.listdir("../")

['test1', 'train', 'sampleSubmission.csv']


['lib', 'input', 'config', 'working']

In [3]:
# The dataset contains images of Cats and Dogs. A separate folder is made for cat and dog images and the 
# label of the image is found out by looking at the file name.
if not os.path.exists("../train"):
    os.mkdir("../train")
if not os.path.exists("../train/dog"):
    os.mkdir("../train/dog")
if not os.path.exists("../train/cat"):
    os.mkdir("../train/cat")
if not os.path.exists("../test"):
    os.mkdir("../test")
if not os.path.exists("../test/dog"):
    os.mkdir("../test/dog")
if not os.path.exists("../test/cat"):
    os.mkdir("../test/cat")

In [4]:
# Training images are moved to 'dog' and 'cat' folders 
for i in tqdm(os.listdir("../input/train/train")):
    if i.split(".")[0] == "dog":
        shutil.copy2(os.path.join("../input/train/train",i),os.path.join("../train/dog",i))
    elif i.split(".")[0] == "cat":
        shutil.copy2(os.path.join("../input/train/train",i),os.path.join("../train/cat",i))        

100%|██████████| 25000/25000 [01:21<00:00, 304.90it/s]


In [5]:
for i in tqdm(os.listdir("../train/dog")[:3000]):
    shutil.move(os.path.join("../train/dog",i),os.path.join("../test/dog",i)) 
for i in tqdm(os.listdir("../train/cat")[:3000]):
    shutil.move(os.path.join("../train/cat",i),os.path.join("../test/cat",i)) 

100%|██████████| 3000/3000 [00:00<00:00, 35387.41it/s]


In [6]:
os.listdir("../train")

['dog', 'cat']

In [7]:
# Training data set fed to the Convolutional Neural Network model
trainflow = ImageDataGenerator()
traindata = trainflow.flow_from_directory(directory="../train",target_size=(224,224))

Found 19000 images belonging to 2 classes.


In [8]:
# Test data set fed to the Convolutional Neural Network model
testflow = ImageDataGenerator()
testdata = testflow.flow_from_directory(directory="../test", target_size=(224,224))

Found 6000 images belonging to 2 classes.


### VGG 16 is a 16 layer Convolutional Neural Network and it uses 3x3 convolutional layers. The later layers have more depth than earlier layers because they have to learn more complex features. After  each convolutional layer, max pooling layer is used because it reduces the dimension of the layer by half which reduces the complexity and increases the computational speed.

<img src="images/1.jpeg">

<img src="images/2.png">

In [9]:
# VGG16 trained on Imagenet dataset is used as a source model for transfer learning
cnnmodel = VGG16(weights='imagenet', include_top=True)

553467904/553467096 [==============================] - 7s 0us/step


In [10]:
cnnmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
# The initial layers are not trained because we are using the same weights that performed well on imagenet dataset 
# because the target dataset is similar to source dataset.
for layers in (cnnmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

In [12]:
A= cnnmodel.layers[-2].output

In [13]:
# The output layer is changed from 1000 neurons to 2 neurons because there are 2 labels, 'cat' and 'dog'
output = Dense(2, activation="softmax")(A)

In [14]:
cdmodel = Model(input = cnnmodel.input, output = output)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [15]:
cdmodel.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [16]:
cdmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [17]:
checkpoint = ModelCheckpoint("cdmodel_1.h5", monitor='val_acc', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1)


In [18]:
hist = cdmodel.fit_generator(generator= traindata, epochs= 10, validation_data= testdata, callbacks=[checkpoint,es])

Epoch 1/10
594/594 [==============================] - 96s 162ms/step - loss: 0.0971 - accuracy: 0.9643 - val_loss: 0.3984 - val_accuracy: 0.9705
Epoch 2/10
  1/594 [..............................] - ETA: 1:03 - loss: 0.0017 - accuracy: 1.0000

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


594/594 [==============================] - 90s 152ms/step - loss: 0.0157 - accuracy: 0.9959 - val_loss: 0.0081 - val_accuracy: 0.9713
Epoch 3/10
594/594 [==============================] - 90s 151ms/step - loss: 0.0055 - accuracy: 0.9996 - val_loss: 0.0991 - val_accuracy: 0.9717
Epoch 4/10
594/594 [==============================] - 90s 152ms/step - loss: 0.0032 - accuracy: 0.9999 - val_loss: 0.0985 - val_accuracy: 0.9725
Epoch 5/10
594/594 [==============================] - 92s 154ms/step - loss: 0.0024 - accuracy: 0.9999 - val_loss: 2.5565e-04 - val_accuracy: 0.9728
Epoch 6/10
594/594 [==============================] - 90s 152ms/step - loss: 0.0018 - accuracy: 0.9999 - val_loss: 0.0488 - val_accuracy: 0.9730
Epoch 7/10
594/594 [==============================] - 90s 152ms/step - loss: 0.0015 - accuracy: 0.9999 - val_loss: 0.0604 - val_accuracy: 0.9728
Epoch 8/10
594/594 [==============================] - 89s 150ms/step - loss: 0.0013 - accuracy: 0.9999 - val_loss: 0.0891 - val_accuracy:

In [20]:
cdmodel.save_weights("cdmodel.h5")